<a href="https://colab.research.google.com/github/RG2806/ai_spotle/blob/master/CNN-82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/RG2806/ai_spotle.git

Cloning into 'ai_spotle'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 27 (delta 13), reused 2 (delta 0), pack-reused 0
Unpacking objects: 100% (27/27), done.


In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.models import Sequential #Initialise our neural network model as a sequential network
from keras.layers import Conv2D #Convolution operation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation#Applies activation function
from keras.layers import Dropout#Prevents overfitting by randomly converting few outputs to zero
from keras.layers import MaxPooling2D # Maxpooling function
from keras.layers import Flatten # Converting 2D arrays into a 1D linear vector
from keras.layers import Dense # Regular fully connected neural network
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score

import pandas as pd
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, GlobalAveragePooling2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import np_utils
import keras.backend as k
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.regularizers import l2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [78]:
def load_data(dataset_path):
  classes = ['Fear', 'Happy', 'Sad']  #We will be dealing with seven different types of emotions.
  data = []
  labels =[]
  with open(dataset_path, 'r') as file:
      for line_no, line in enumerate(file.readlines()):
          if 0 < line_no <= 35887:
            line = line.split(',')
            image_data = np.asarray([int(x) for x in line[1:]]).reshape(48,48)#Creating a list out of the string then converting it into a 2-Dimensional numpy array.
            image_data =image_data.astype(np.float32)/255.0
            data.append(image_data)
            labels.append(classes.index(line[0]))
            data.append(cv2.flip(image_data, 1))
            labels.append(classes.index(line[0]))
  data = np.expand_dims(data, -1)   
  labels = to_categorical(labels, num_classes = 3)
    
  return np.array(data), np.array(labels)

In [79]:
dataset_path = "/content/ai_spotle/aithon2020_level2_traning.csv" 
train_data, train_labels = load_data(dataset_path)
#train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = test_size,random_state = seed)

print("Number of images in Training set:", len(train_data))

Number of images in Training set: 21634


In [81]:
train_data.shape

(21634, 48, 48, 1)

In [82]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=101,shuffle=True)
#y_train = np_utils.to_categorical(y_train, 3)
#y_test = np_utils.to_categorical(y_test, 3)

(array([0, 1, 2]), array([4423, 7668, 5216]))

In [83]:
num_features = 64
width, height = 48, 48
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_28 (Batc (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_29 (Batc (None, 23, 23, 128)      

In [84]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint('/content/input/model.h5', monitor='val_accuracy', verbose=1, save_best_only=True,mode='max')
checkpointer2= ModelCheckpoint('/content/input2/model2.h5', monitor='val_loss', verbose=1, save_best_only=True,mode='min')
callbacks = [lr_reducer, checkpointer,checkpointer2]

In [85]:
# With Augmentation
bs = 64
epochs = 100

aug = ImageDataGenerator(rotation_range=20, 
                         width_shift_range=0.1, 
                         height_shift_range=0.1,  
                         fill_mode="nearest")

H = model.fit_generator(aug.flow(x_train, y_train, batch_size=bs),
                        validation_data=(x_test, y_test), 
                        steps_per_epoch=len(x_train)//bs,
                        callbacks=callbacks,
                        shuffle=True,
                        epochs=epochs)

Epoch 1/100
270/270 [==============================] - ETA: 0s - loss: 1.2850 - accuracy: 0.3954
Epoch 00001: val_accuracy improved from -inf to 0.45066, saving model to /content/input/model.h5

Epoch 00001: val_loss improved from inf to 1.08577, saving model to /content/input2/model2.h5
270/270 [==============================] - 27s 101ms/step - loss: 1.2850 - accuracy: 0.3954 - val_loss: 1.0858 - val_accuracy: 0.4507
Epoch 2/100
270/270 [==============================] - ETA: 0s - loss: 1.0915 - accuracy: 0.4373
Epoch 00002: val_accuracy did not improve from 0.45066

Epoch 00002: val_loss improved from 1.08577 to 1.07161, saving model to /content/input2/model2.h5
270/270 [==============================] - 27s 99ms/step - loss: 1.0915 - accuracy: 0.4373 - val_loss: 1.0716 - val_accuracy: 0.4507
Epoch 3/100
270/270 [==============================] - ETA: 0s - loss: 1.0733 - accuracy: 0.4481
Epoch 00003: val_accuracy did not improve from 0.45066

Epoch 00003: val_loss improved from 1.07

In [46]:
scores = model.evaluate(x_test, y_test, batch_size=bs)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

68/68 [==============================] - 2s 28ms/step - loss: 0.4904 - accuracy: 0.8197
Loss: 0.49035680294036865
Accuracy: 0.8197365403175354


In [90]:
import sklearn.metrics as metrics
y_pred=model.predict(x_test)
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(metrics.classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.72      0.65      0.68      1051
           1       0.93      0.94      0.93      1950
           2       0.74      0.80      0.77      1326

    accuracy                           0.82      4327
   macro avg       0.80      0.79      0.79      4327
weighted avg       0.82      0.82      0.82      4327



In [47]:
model.save_weights("/content/model.h5")
print("Saved model to disk")

Saved model to disk


In [50]:
import os
from tensorflow import keras
model = keras.models.load_model('/content/input/model.h5')
def getData():
    # images are 48x48
    # N = 35887
    X = []
    Y=[]
    Z=[]
    first = True
    for count, filename in enumerate(os.listdir("/content/drive/My Drive/test/0")):
          img = image.load_img("/content/drive/My Drive/test/0/"+filename, grayscale=True, target_size=(48, 48))
          #show_img=image.load_img('/content/hawon.jpg', grayscale=False, target_size=(200, 200))
          x = image.img_to_array(img)
          x = np.expand_dims(x, axis = 0)
          x = np.array(x , dtype = np.uint8)
          X.append(np.argmax(model.predict(x)))
    for count, filename in enumerate(os.listdir("/content/drive/My Drive/test/1")):
          img = image.load_img("/content/drive/My Drive/test/1/"+filename, grayscale=True, target_size=(48, 48))
          #show_img=image.load_img('/content/hawon.jpg', grayscale=False, target_size=(200, 200))
          x = image.img_to_array(img)
          x = np.expand_dims(x, axis = 0)
          x = np.array(x , dtype = np.uint8)
          Y.append(np.argmax(model.predict(x)))
    for count, filename in enumerate(os.listdir("/content/drive/My Drive/test/2")):
          img = image.load_img("/content/drive/My Drive/test/2/"+filename, grayscale=True, target_size=(48, 48))
          #show_img=image.load_img('/content/hawon.jpg', grayscale=False, target_size=(200, 200))
          x = image.img_to_array(img)
          x = np.expand_dims(x, axis = 0)
          x = np.array(x , dtype = np.uint8)
          Z.append(np.argmax(model.predict(x)))
    return X,Y,Z

X,Y,Z=getData()
count1=0
count2=0
count3=0
for i in X:
    if i==0:
      count1=count1+1
for i in Y:
    if i==1:
      count2=count2+1
for i in Z:
    if i==2:
      count3=count3+1
print(count1/len(X))
print(count2/len(Y))
print(count3/len(Z))

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


0.8888888888888888
0.027777777777777776
0.05555555555555555


In [ ]:
0.75
0.16666666666666666
0.16666666666666666

In [ ]:
0.8055555555555556
0.25
0.08333333333333333

In [ ]:
0.9166666666666666
0.027777777777777776
0.1388888888888889

In [ ]:
show_img=image.load_img('/content/hawon.jpg', grayscale=False, target_size=(200, 200))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
x = np.array(x , dtype = np.uint8)
np.argmax(model.predict(x))